# Imports

In [59]:
import re
import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion
import importlib.util
from pathlib import Path

# Parse Input

In [60]:
class InputData:
    def __init__(self, input_text: Path):
        self.parse(input_text)
    
    def __repr__(self) -> str:
        return f"function_name: {self.function_name}\nsignature: {self.signature}\ndescription: {self.description}\nexamples: {self.examples}\ntemperatures: {self.temperatures}\nnum: {self.num}\nunit_tests: {self.unit_tests}"
    
    def parse(self, input_text: Path):
        with open(input_text, 'r') as f:
            self.text = f.readlines()
        self.text = [line.strip().removesuffix(";") for line in self.text if line.strip() != '']
        self.function_name = self.text[0]
        self.description = ""
        for i, line in enumerate(self.text[2:]):
            if '"""' in line:
                break
            if 'signature' in line:
                self.signature = line
            else:
                self.description += f"{line} "
        
        next_part = self.text[i + 1:]
        self.examples = []
        for i, line in enumerate(next_part):
            if "example:" in line:
                self.examples.append(line)
            if "temperatures:" in line:
                self.temperatures = line.split(":")[1].split(",")
                self.temperatures = [float(temp) for temp in self.temperatures]
            if "num" in line:
                self.num = int(line.split('=')[1])
            if "unit_tests" in line:
                self.unit_tests = next_part[i + 1:]
                break
            
            


In [61]:
input_text = Path('input.txt')
input_data = InputData(input_text)
input_data

function_name: sqrt_list
signature: signature: sqrt_list(l:list) -> float
description: A function named sqrt_list that takes a list of integer and returns the square root of the sum of that list, to two decimal places. 
examples: ['example: sqrt_list([1,7,8]) -> 4.0']
temperatures: [0.0, 0.8]
num: 3
unit_tests: ['sqrt_list([6,18,7,5]) -> 6.0', 'sqrt_list([50,2,33,8,4,3,10]) -> 10.48']

# Intigrate model

In [3]:
AZURE_OPENAI_API_KEY="e865ffe4387f416ea57d7394ee09fb2e"
AZURE_OPENAI_ENDPOINT="https://openaifor3267.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2023-12-01-preview"
model = "gpt-35-16k"

In [ ]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT
)

In [4]:
prompt = "Once upon a time"

In [ ]:
def sent_prompt(prompt: str, temp: float, num_of_samples: int) -> ChatCompletion:
    response = client.chat.completions.create(
        model=model,
        temperature=temp,
        n=num_of_samples,
        max_tokens=100,
        messages=[
            {"role": "system", "content": "Act as experienced python developer"},
            {"role": "user", "content": prompt}
        ]
    )
    return response

In [ ]:
response = sent_prompt(prompt, 0.5, 1)

In [5]:
genreated_file = 'generated.py'
function_name = "sqrt_list"

In [7]:
# Load the module from the file
spec = importlib.util.spec_from_file_location("generated", genreated_file)
generated = importlib.util.module_from_spec(spec)
spec.loader.exec_module(generated)

# Access the function from the dynamically generated module
dynamic_function = getattr(generated, function_name)

# Now you can call the function
#result = dynamic_function([6, 18, 7, 5])


In [13]:
l = eval("6, 2")

In [15]:
l[1]

2

In [ ]:
result

In [ ]:
with open(genreated_file, "w") as f:
    s = """
    def sqrt_list(l):
        return [x**0.5 for x in l]
    """.strip()
    f.write(s)

In [ ]:
result

In [ ]:
with open(genreated_file, "w") as f:
    s = """
    def sqrt_list(l):
        return [x**0.5 for x in l]
    """.strip()
    f.write(s)

In [ ]:
from genreated_file import sqrt_list

In [ ]:
eval("sqrt_list([6,18,7,5])")